In [32]:
import csv
import random
import importlib
from openai import OpenAI

import constants # Required for the following line if kernel is restarted
importlib.reload(constants) # Else the old key value is retained
from constants import openai_api

client = OpenAI(api_key=openai_api)
CSV_FILE_NAME = "sample_output_r10_c3.csv"

In [34]:
def attack_with_gpt():
    with open(CSV_FILE_NAME, mode='r') as file:
        reader = csv.reader(file)
        rows = list(reader)

    headers = rows[0]
    data = rows[1:]
    
    random.shuffle(data)
    shuffled_rows = [headers] + data
    
    formatted_data = "\n".join([", ".join(row) for row in shuffled_rows])

    prompt = f"""
    You are a query analysis assistant. Below is a CSV file containing rows of sequential queries.
    Each row corresponds to a different topic, and each column represents a sequential query flow.
    Only one of the rows is a geniune query flow made by a human user, the rest are AI generated
    to twart any adversary from figuring out what the actual search intention is. Your task is to 
    analyze the queries in each row and identify which one is the real, human-generated query. 
    Rank your predictions and return the top 10. Return them in a JSON format where the key is 
    the prediction rank and the value is the row index of the original query.

    CSV Data:
    {formatted_data}
    """

    response = client.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=[
            {"role": "system", "content": "You are a query analysis assistant."},
            {"role": "user", "content": prompt}
        ],
        response_format={ "type": "json_object" },
        temperature=0.7,
        seed=random.randint(0, 1000000)
    )

    result = response.choices[0].message.content
    return result

def evaluate_multiple_times(csv_path, api_key, iterations):
    with open(csv_path, mode='r') as file:
        reader = csv.reader(file)
        rows = list(reader)

    headers = rows[0]
    data = rows[1:]

    for i in range(iterations):
        random.shuffle(data)
        shuffled_csv_path = f"shuffled_{i}_{csv_path}"

        with open(shuffled_csv_path, mode='w', newline='') as shuffled_file:
            writer = csv.writer(shuffled_file)
            writer.writerow(headers)
            writer.writerows(data)

        print(f"Iteration {i + 1}: Shuffled rows and analyzing...")
        gpt_analysis = analyze_csv_with_gpt(shuffled_csv_path, api_key)
        evaluate_results(gpt_analysis)

def evaluate_results(gpt_results):
    import json
    results = json.loads(gpt_results)

    for rank, row_index in results.items():
        if row_index == 0:
            print(f"Original query was detected as rank {rank}")
            return int(rank)
    return -1
            

N_ATTACKS = 10
success_rank_sum = 0
success_count_sum = 0
for i in range(N_ATTACKS):
    gpt_result = attack_with_gpt()
    #print(gpt_result)
    rank = evaluate_results(gpt_result)
    if rank > 0:
        success_rank_sum += rank
        success_count_sum += 1
print(f"Average success rank: {success_rank_sum / success_count_sum}")
print(f"Average success count: {success_count_sum / N_ATTACKS}")

Original query was detected as rank 1
Original query was detected as rank 2
Original query was detected as rank 1
Original query was detected as rank 6
Original query was detected as rank 8
Original query was detected as rank 1
Original query was detected as rank 7
Original query was detected as rank 4
Original query was detected as rank 1
Average success rank: 3.4444444444444446
Average success count: 0.9
